In [1]:
#Generación de archivos fasta para las 30 especies
import sys
sys.path.append('./libs')
import util_bd, util_fasta, util_caracteristicas
import os
import shutil
from sklearn.externals.joblib import Parallel, delayed, dump
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import csv

#generación de bd diamond
#script = "~/anaconda3/bin/diamond makedb --in ./lncRNApred/data/uniprot_sprot.fasta -d ./lncRNApred/data/swissprot.dmnd"
#os.system(script)

def obtener_especies():
    query = "SELECT * FROM (SELECT CONVERT(@row_number:=@row_number + 1, UNSIGNED) AS orden, m.id_especie, m.especie FROM especies_seleccionadas s JOIN maestra_especies m ON s.especie = m.especie, (SELECT @row_number:=0) AS rn ORDER BY m.id_especie) a ORDER BY 1"
    return util_bd.resultados_query(query)
        
def generar_fasta_especie(row_especie):
    # lncRNA
    query = "SELECT cod_secuencia, secuencia FROM secuencias WHERE flg_pct = 0 AND flg_seleccionado = 1 AND id_especie = " + str(row_especie[1]) + " ORDER BY cod_secuencia"
    secuencias = util_bd.resultados_query(query)
    util_fasta.generar_fasta(secuencias, "./lncRNApred/data/lncRNA.fa")
    # PCT
    query = "SELECT cod_secuencia, secuencia FROM secuencias WHERE flg_pct = 1 AND flg_seleccionado = 1 AND id_especie = " + str(row_especie[1]) + " ORDER BY cod_secuencia"
    secuencias = util_bd.resultados_query(query)
    util_fasta.generar_fasta(secuencias, "./lncRNApred/data/PCT.fa")
    # lncRNA+PCT
    query = "SELECT cod_secuencia, secuencia FROM secuencias WHERE flg_seleccionado = 1 AND id_especie = " + str(row_especie[1]) + " ORDER BY flg_pct ASC, cod_secuencia"
    secuencias = util_bd.resultados_query(query)
    util_fasta.generar_fasta(secuencias, "./lncRNApred/data/transcritos.fa")

def ejecutar_scripts():
    #diamond
    script = "~/anaconda3/bin/diamond blastx -d ./lncRNApred/data/swissprot.dmnd -q ./lncRNApred/data/transcritos.fa -o ./lncRNApred/data/transcritos.dmnd -e 0.001 -k 5 --matrix BLOSUM62 --gapopen 11 --gapextend 1 --more-sensitive -f 6 qseqid pident length qframe qstart qend sstart send evalue bitscore"
    os.system(script)
    #cpat
    script = "~/anaconda3/bin/cpat.py -g ./lncRNApred/data/transcritos.fa -o ./lncRNApred/data/transcritos.cpat -x ./lncRNApred/crema/cpat_models/ath_hexamer.txt -d ./lncRNApred/crema/cpat_models/ath_logit.RData"
    os.system(script)
    #lncRNApred
    os.chdir('./lncRNApred/results/')
    script = "python3 ../crema/bin/predict.py -f ../data/transcritos.fa -c ../data/transcritos.cpat -d ../data/transcritos.dmnd"
    os.system(script)
    os.chdir('../../')

def obtener_resultados(num_especie):
    transcritos_lncRNA = {k.upper():"" for k,_ in util_fasta.leer_fasta("./lncRNApred/data/lncRNA.fa").items()}
    transcritos_PCT = {k.upper():"" for k,_ in util_fasta.leer_fasta("./lncRNApred/data/PCT.fa").items()}
    y_true = list()
    y_pred = list()
    with open("./lncRNApred/results/final_ensemble_predictions.csv", "r") as outfile:
        csv_reader = csv.reader(outfile, delimiter=(","))
        next(csv_reader, None) # skip header
        for row in csv_reader:
            if row[0].upper() in transcritos_lncRNA:
                y_true.append(1)
            if row[0].upper() in transcritos_PCT:
                y_true.append(0)
            y_pred.append(int(row[11]))
    print("ESPECIE " + str(num_especie) + ": " + str(accuracy_score(y_true, y_pred)))
    matriz_confusion = confusion_matrix(y_true, y_pred)
    print(classification_report(y_true, y_pred, target_names=["lncRNA", "PCT"]))
    tn, fp, fn, tp = matriz_confusion.ravel()
    print(matriz_confusion)
    print("tn:{}, fp:{}, fn:{}, tp:{}".format(tn, fp, fn, tp))
    
def resultados_especie(row_especie):
    generar_fasta_especie(row_especie)
    ejecutar_scripts()
    obtener_resultados(row_especie[0])
    
def preparar_data_maix():
    shutil.copy("./lncRNApred/data/zea_mays_lncRNA.fa", "./lncRNApred/data/lncRNA.fa")
    shutil.copy("./lncRNApred/data/zea_mays_PCT.fa", "./lncRNApred/data/PCT.fa")
    shutil.copy("./lncRNApred/data/zea_mays.fa", "./lncRNApred/data/PCT.fa")

#EJECUCION 30 ESPECIES
for row_especie in obtener_especies():
    resultados_especie(row_especie)

#EJECUCION MAIX
ejecutar_scripts()
obtener_resultados("ZEA MAYS")

print("Proceso finalizado")

ESPECIE 1: 0.36549429657794674
              precision    recall  f1-score   support

      lncRNA       0.42      0.73      0.53      4208
         PCT       0.01      0.00      0.01      4208

   micro avg       0.37      0.37      0.37      8416
   macro avg       0.22      0.37      0.27      8416
weighted avg       0.22      0.37      0.27      8416

[[3062 1146]
 [4194   14]]
tn:3062, fp:1146, fn:4194, tp:14
ESPECIE 2: 0.682865969581749
              precision    recall  f1-score   support

      lncRNA       0.63      0.91      0.74      4208
         PCT       0.83      0.46      0.59      4208

   micro avg       0.68      0.68      0.68      8416
   macro avg       0.73      0.68      0.67      8416
weighted avg       0.73      0.68      0.67      8416

[[3828  380]
 [2289 1919]]
tn:3828, fp:380, fn:2289, tp:1919
ESPECIE 3: 0.6432984790874525
              precision    recall  f1-score   support

      lncRNA       0.59      0.90      0.72      4208
         PCT       0.79   

ESPECIE 21: 0.5497861216730038
              precision    recall  f1-score   support

      lncRNA       0.54      0.71      0.61      4208
         PCT       0.57      0.39      0.46      4208

   micro avg       0.55      0.55      0.55      8416
   macro avg       0.56      0.55      0.54      8416
weighted avg       0.56      0.55      0.54      8416

[[2989 1219]
 [2570 1638]]
tn:2989, fp:1219, fn:2570, tp:1638
ESPECIE 22: 0.4610266159695818
              precision    recall  f1-score   support

      lncRNA       0.48      0.78      0.59      4208
         PCT       0.39      0.14      0.21      4208

   micro avg       0.46      0.46      0.46      8416
   macro avg       0.44      0.46      0.40      8416
weighted avg       0.44      0.46      0.40      8416

[[3270  938]
 [3598  610]]
tn:3270, fp:938, fn:3598, tp:610
ESPECIE 23: 0.6270199619771863
              precision    recall  f1-score   support

      lncRNA       0.59      0.83      0.69      4208
         PCT       0.7